# Bridge (most)

In [1]:
from io import StringIO

In [2]:
class Shape(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @classmethod
    def create_factory(cls):
        shape_factory = {}
        for subclass in cls.__subclasses__():
            shape_factory[subclass.__name__] = subclass
            # recurse if you want to include subsubclasses and so on
            # shape_factory.update(subclass.create_factory())
        return shape_factory
        
class Circle(Shape):
    def __init__(self, x, y, r):
        # super(Circle, self).__init__(x, y)  # Python 2
        super().__init__(x, y)  # Python 3
        self.r = r
        
    def __repr__(self):
        return 'Circle x={} y={} r={}'.format(self.x, self.y, self.r)
    
    def draw(self, drawing):
        drawing.draw_circle(self.x, self.y, self.r)
        
class Rectangle(Shape):
    def __init__(self, x, y, w, h):
        # super(Rectangle, self).__init__(x, y)  # Python 2
        super().__init__(x, y)  # Python 3
        self.w = w
        self.h = h
        
    def __repr__(self):
        return 'Rectangle x={} y={} w={} h={}'.format(self.x, self.y, self.w, self.h)
    
    def draw(self, drawing):
        x, y, w, h = self.x, self.y, self.w, self.h
        drawing.draw_line(x  , y  , x+w, y  )
        drawing.draw_line(x  , y  , x  , y+h)
        drawing.draw_line(x+w, y  , x+w, y+h)
        drawing.draw_line(x  , y+h, x+w, y+h)
    
class Square(Shape):
    def __new__(cls, x, y, a):
        return Rectangle(x, y, a, a)

In [3]:
raw_shapes = '''
Circle 15 10 14
Rectangle 30 30 100 150
Circle 40 20 5
Square 30 100 20
'''

In [4]:
class Drawing(object):
    def __init__(self, shapes):
        self._shapes = shapes
        
    def __repr__(self):
        return '<Drawing {}>'.format(str(self._shapes))
        
    @classmethod
    def from_stream(cls, stream):
        shape_factory = Shape.create_factory()
        shapes = []
        for line in stream:
            line = line.strip()
            if not line:
                continue
            # splited = line.split()  # Python 2
            # shape_name = splited[0]  # Python 2
            # parameters = splited[1:]  # Python 2
            shape_name, *parameters = line.split()  # Python 3
            parameters = map(int, parameters)
            try:
                shape_creator = shape_factory[shape_name]
            except KeyError:
                raise TypeError
            else:
                shape = shape_creator(*parameters)
            shapes.append(shape)
        return cls(shapes)
    
    def draw(self):
        for shape in self._shapes:
            shape.draw(self)
            
    def draw_line(self, x1, y1, x2, y2):
        raise NotImplemented
        
    def draw_circle(self, x, y, r):
        raise NotImplemented

In [5]:
try:
    import tkinter  # Python 3
except ImportError:
    import Tkinter as tkinter  # Python 2

class TKinterDrawing(Drawing):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._window = tkinter.Tk()
        
    def draw_line(self, x1, y1, x2, y2):
        self._canvas.create_line(x1, 200-y1, x2, 200-y2)
    
    def draw_circle(self, x, y, r):
        self._canvas.create_oval((x-r, 200-(y-r), x+r, 200-(y+r)))
    
    def draw(self):
        root = tkinter.Tk()
        frame = tkinter.Frame(master=root)
        self._canvas = tkinter.Canvas(frame)
        self._canvas.pack()
        frame.pack()
        root.geometry('400x400')
        super().draw()
        root.mainloop()

drawing = TKinterDrawing.from_stream(StringIO(raw_shapes))

drawing.draw()

### Rozwiązanie

In [6]:
import turtle

class TurtleDrawing(Drawing):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        turtle.resetscreen()
        turtle.penup()
        
    def draw_line(self, x1, y1, x2, y2):
        turtle.goto(x1, y1)
        turtle.pendown()
        turtle.goto(x2, y2)
        turtle.penup()
    
    def draw_circle(self, x, y, r):
        turtle.goto(x, y-r)
        turtle.pendown()
        turtle.circle(r)
        turtle.penup()

drawing = TurtleDrawing.from_stream(StringIO(raw_shapes))

drawing.draw()